This is an example G-Colab notebook on how to generate a vector store. Here we'll be looking at the `textbooks` files from the MedRAG databases.

In [ ]:
!pip install -r /content/requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.9/704.9 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
!pip install unstructured -q
!pip install jq -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.6/657.6 kB 9.6 MB/s eta 0:00:00


In [ ]:
!pip install faiss-gpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.9 MB/s eta 0:00:00


In [ ]:
#this loads all the jsonl files for the textbooks from the MedRAG database
!git clone https://huggingface.co/datasets/MedRAG/textbooks

Cloning into 'textbooks'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 31 (delta 1), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (31/31), 7.62 KiB | 976.00 KiB/s, done.
Filtering content: 100% (18/18), 201.76 MiB | 52.11 MiB/s, done.


In [ ]:
import os
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders import JSONLoader
from langchain_text_splitters import RecursiveJsonSplitter
from langchain_community.vectorstores import FAISS

from langchain.embeddings.huggingface import HuggingFaceEmbeddings



# Directory containing JSONL files
directory = '/content/textbooks/chunk/'

# Load and process all JSONL files
documents = []
splitter = RecursiveJsonSplitter()

for filename in os.listdir(directory):
    if filename.endswith('.jsonl'):
      jsonl_loader = JSONLoader(directory+filename, jq_schema = '.content', json_lines =True)
      json_data = jsonl_loader.load()
      documents.extend(json_data)

      # loader = UnstructuredFileLoader(file_path=filename)
      # # Initialize the splitter with a maximum chunk size
      # # Recursively split json data
      # json_chunks = splitter.split_json(json_data=loader.load())
      # for chunk in json_chunks:
      #   documents.append(chunk)

#to create all the embeddings for the vector store, using the popular all-MiniLM-L6-v2 embedding model
embeddings= HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create the vector store
vector_store = FAISS.from_documents(documents, embeddings)

In [ ]:
# Perform a similarity search
docs_and_scores = vector_store.similarity_search_with_score("What is CPR?")


docs_and_scores[0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(Document(page_content='Basic life support, more popularly known as CPR, is intended to maintain organ perfusion until definitive interventions can be instituted. The initial and primary element of CPR is maintenance of perfusion until spontaneous circulation can be restored. Closed chest cardiac compression maintains a pump function by sequential filling and emptying of the chambers, with competent valves maintaining forward direction of flow. The palm of one hand is placed over the lower sternum, with the heel of the other resting on the dorsum of the lower hand. The sternum is depressed, with the arms remaining straight, at a rate of 100 per minute. Sufficient force is applied to depress the sternum 4–5 cm, and relaxation is abrupt.', metadata={'source': '/content/textbooks/chunk/InternalMed_Harrison.jsonl', 'seq_num': 21068}),
 0.795031)

In [ ]:
#save the vector store
vector_store.save_local("faiss_index_full")


In [ ]:
#loading the vector store
new_db = FAISS.load_local('/content/faiss_index_full', embeddings, allow_dangerous_deserialization=True)

In [ ]:
# Perform a similarity search using the loaded vector store
docs_and_scores = new_db.similarity_search_with_score("I have a headache and a fever what should i do?")


# Print the results
for doc,  score in docs_and_scores:
    print(f"Document: {doc.page_content}")
    print(f'meta: {doc.metadata}')
    print(f"Score: {score}")
    print()


Document: Conditions that can cause headache, fever, focal neurologic signs, and seizure activity include brain abscess, subdural empyema, bacterial meningitis, viral meningoencephalitis, superior sagittal sinus thrombosis, and acute disseminated encephalomyelitis. When fever is absent, primary and metastatic brain tumors become the major differential diagnosis. Less commonly, cerebral infarction or hematoma can have an MRI or CT appearance resembling brain abscess.
meta: {'source': '/content/textbooks/chunk/InternalMed_Harrison.jsonl', 'seq_num': 11102}
Score: 0.9581701755523682

Document: myoclonus) is a special issue, which is discussed in Chap. 4. Its treatment usually requires the use of multiple medications. Fever is treated with antipyretics or a cooling blanket combined with neuromuscular paralyzing agents.
meta: {'source': '/content/textbooks/chunk/Neurology_Adams.jsonl', 'seq_num': 9088}
Score: 0.9673322439193726

Document: Most patients seek medical care for sore throat and 